In [2]:
# import library you use
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd

# Training phase

### Load Data

In [3]:
# load train.csv and training-label.csv
dataset = pd.read_csv('train.csv')
label = pd.read_csv('train_label.csv')

### Data Preprocessing

In [4]:
# drop created_time column
dataset=dataset.drop("created_time", axis=1)

In [5]:
# Count whether each mac_hash has passed through these 14 locations, and generate a training dataframe
newdataset = pd.DataFrame(columns = ['mac_hash'])
newdataset2 = pd.DataFrame([{'mac_hash' : '10f6ad2e3fa3b3da8ceb3fbd95658681'}])
newdataset = pd.concat([newdataset, newdataset2], ignore_index=True)
for index, row in dataset.iterrows():
    if (row['mac_hash']!=newdataset.iloc[-1].mac_hash):
        appenddata = pd.DataFrame([{'mac_hash' : row['mac_hash']}])
        newdataset = pd.concat([newdataset, appenddata], ignore_index=True)

for index, row in dataset.iterrows():
    number = newdataset.index[newdataset['mac_hash']==row['mac_hash']][0]
    columntag=str(row['sniffer_loc'])
    newdataset.at[number, columntag] = '1'

newdataset=newdataset.loc[:,['mac_hash','1','2','3','4','5','6','7','8','9','10','11','12','13','14']]
newdataset=newdataset.fillna('0')
print(newdataset)

                              mac_hash  1  2  3  4  5  6  7  8  9 10 11 12 13  \
0     10f6ad2e3fa3b3da8ceb3fbd95658681  0  1  1  1  1  1  1  0  1  1  1  1  0   
1     5272841b630d74290ec129c64dd6378b  0  0  0  0  0  0  0  0  0  0  0  1  0   
2     8ab74d094708a27c9660e897b93a50f9  1  1  1  1  1  1  1  1  1  1  1  1  1   
3     3d00887dd6e574dd63974bc6c69b23b0  0  1  1  1  0  0  1  0  1  0  1  0  0   
4     6461fce7f6f184401c961fda75069262  0  1  1  0  1  0  0  0  0  0  0  1  0   
...                                ... .. .. .. .. .. .. .. .. .. .. .. .. ..   
5563  82498aa0a931e2f37fd29998a77c6dd0  0  0  0  0  0  0  0  0  0  0  0  1  0   
5564  656ddc677dcb8661ff5d782742fcecfd  0  0  1  0  0  0  0  0  0  0  0  0  0   
5565  98d389a4b2f378f8217a02416aaa9349  1  0  0  0  0  0  0  0  0  0  0  0  0   
5566  1eff5eae7401ee128798660ea6b6f09b  0  1  0  1  0  1  0  0  0  0  1  0  0   
5567  13a3e10f950d8050ab5437cf30a3c2c6  1  1  1  1  1  1  1  1  1  0  1  0  0   

     14  
0     1  
1     0

In [6]:
# Add label column
labeldata=pd.merge(newdataset,label)
finaldata=labeldata.sort_values(by=['mac_hash'], ascending=True)
finaldata = finaldata.reset_index(drop=True)
print(finaldata)

                              mac_hash  1  2  3  4  5  6  7  8  9 10 11 12 13  \
0     0003ae8541d0e925fcee242287e2ad27  0  0  0  0  0  0  0  0  0  0  1  0  0   
1     00078611037990f7f36b722f22595fe7  0  1  1  1  1  1  0  1  0  1  1  0  0   
2     000ce31739b333d931813de403156844  1  1  1  1  1  1  1  1  1  1  1  1  1   
3     0027240b2aeb1f7f3c053c073cafb452  0  1  0  1  1  1  1  1  1  1  1  1  1   
4     0028a89964416dc9a8a66cc7fe25ef4e  0  1  1  1  0  1  0  0  1  1  1  1  1   
...                                ... .. .. .. .. .. .. .. .. .. .. .. .. ..   
5563  ffbf58ded2902366bcae7aa5e468556c  0  1  1  1  1  1  0  1  1  1  1  1  0   
5564  ffc343f03f0c9dbd06439ddff42922c0  0  0  1  0  0  0  0  0  0  0  0  0  0   
5565  ffca215be2cb1505d931bd162da60104  0  1  1  1  1  1  0  1  0  1  0  1  1   
5566  ffd7be58ee3c3fb47530ae7e4f251f88  0  0  0  0  1  0  0  0  0  0  0  0  0   
5567  fff2ca0333532d6348d5b4fa39028dd4  0  0  1  0  0  0  0  0  1  1  1  1  1   

     14  label  
0     0   

### Training

In [7]:
# X: feature from training dataframe
X=finaldata[finaldata.columns[1:15]]
X = X.astype('int64')

In [8]:
# Y: label from training dataframe
# you need to change your label from categorical encoding to one-hot encoding
from sklearn.preprocessing import OneHotEncoder
Y = finaldata[finaldata.columns[15]]
onehotencoder = OneHotEncoder()
Y=onehotencoder.fit_transform(Y.values.reshape(-1, 1)).toarray()
pd.DataFrame(Y)

0    1    2    3    4
0     1.0  0.0  0.0  0.0  0.0
1     0.0  1.0  0.0  0.0  0.0
2     0.0  0.0  0.0  1.0  0.0
3     0.0  0.0  0.0  0.0  1.0
4     0.0  0.0  0.0  0.0  1.0
...   ...  ...  ...  ...  ...
5563  0.0  0.0  0.0  0.0  1.0
5564  1.0  0.0  0.0  0.0  0.0
5565  0.0  0.0  0.0  0.0  1.0
5566  1.0  0.0  0.0  0.0  0.0
5567  0.0  0.0  1.0  0.0  0.0

[5568 rows x 5 columns]

In [9]:
# split your dataset as training dataset and validation dataset
# using train_test_split function,
### training dataset : validation dataset = 8 : 2
### random_state = 0
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, Y, test_size=0.2, random_state=0)

In [10]:
from keras.models import Sequential
from keras.layers.core import Dense,Activation
from keras.optimizers import Adam
from sklearn import preprocessing
# build DNN model
def build_model():
#Write the code
    #contruct model
    model = Sequential()
    #stack model
    model.add(Dense(input_dim=14,units=40)) #The number of columns for each data
    model.add(Activation("relu"))
    model.add(Dense(units=100))
    model.add(Activation("relu"))
    model.add(Dense(units=10))
    model.add(Activation("relu"))
    model.add(Dense(units=5))
    model.add(Activation("Softmax"))
    model.summary()
    return model

In [11]:
# start train model
model = build_model()
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['acc'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 40)                600       
                                                                 
 activation (Activation)     (None, 40)                0         
                                                                 
 dense_1 (Dense)             (None, 100)               4100      
                                                                 
 activation_1 (Activation)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
 activation_2 (Activation)   (None, 10)                0         
                                                                 
 dense_3 (Dense)             (None, 5)                 5

In [12]:
# validate your model using validation dataset
# After prediction, you need to change your result and label from one-hot encoding to categorical encoding
train_history = model.fit(x=X_train,y=y_train,
                          validation_split=0.2,batch_size=30,epochs=30)

minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaledFeatures = minmax_scale.fit_transform(X_train)

score = model.evaluate(x=scaledFeatures, y=y_train)
print ('\nTrain Loss:', score[0])
print ('\nTrain Acc:', score[1])

Epoch 1/30
119/119 [==============================] - 1s 2ms/step - loss: 1.0642 - acc: 0.6520 - val_loss: 0.4688 - val_acc: 0.8777
Epoch 2/30
119/119 [==============================] - 0s 1ms/step - loss: 0.2531 - acc: 0.9413 - val_loss: 0.1860 - val_acc: 0.9473
Epoch 3/30
119/119 [==============================] - 0s 1ms/step - loss: 0.1397 - acc: 0.9624 - val_loss: 0.1390 - val_acc: 0.9607
Epoch 4/30
119/119 [==============================] - 0s 1ms/step - loss: 0.1124 - acc: 0.9683 - val_loss: 0.1029 - val_acc: 0.9742
Epoch 5/30
119/119 [==============================] - 0s 1ms/step - loss: 0.0895 - acc: 0.9756 - val_loss: 0.0903 - val_acc: 0.9764
Epoch 6/30
119/119 [==============================] - 0s 1ms/step - loss: 0.0822 - acc: 0.9764 - val_loss: 0.0892 - val_acc: 0.9753
Epoch 7/30
119/119 [==============================] - 0s 1ms/step - loss: 0.0760 - acc: 0.9795 - val_loss: 0.0795 - val_acc: 0.9708
Epoch 8/30
119/119 [==============================] - 0s 2ms/step - loss: 0.

In [13]:
#predictedargmax = np.argmax(model.predict(X_train), axis=-1)
#expectedargmax = np.argmax(y_train, axis=-1)
predicted = onehotencoder.inverse_transform(model.predict(X_train))
expected = onehotencoder.inverse_transform(y_train)

140/140 [==============================] - 0s 674us/step


In [14]:
# using sklearn classification_report function to show your validation result
from sklearn.metrics import classification_report
from sklearn import ensemble, metrics

print(metrics.classification_report(expected, predicted))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1723
           1       0.99      1.00      0.99       714
           2       0.99      0.99      0.99       496
           3       1.00      0.96      0.98       517
           4       0.98      0.99      0.99      1004

    accuracy                           0.99      4454
   macro avg       0.99      0.99      0.99      4454
weighted avg       0.99      0.99      0.99      4454



# Testing phase

### Load Data

In [15]:
# load test.csv file as dataframe
testdataset = pd.read_csv('test.csv')

### Data Preprocessing

In [16]:
# Preprocessing is the same as training
def datapreprocess(data):
    data=data.drop("created_time", axis=1)
    newdata = pd.DataFrame(columns = ['mac_hash'])
    newdata2 = pd.DataFrame([{'mac_hash' : 'NaN'}])
    newdata = pd.concat([newdata, newdata2], ignore_index=True)
    newdata = newdata.dropna(axis=0)
    
    for index, row in data.iterrows():
        if (row['mac_hash']!=newdata.iloc[-1].mac_hash):
            appenddata = pd.DataFrame([{'mac_hash' : row['mac_hash']}])
            newdata = pd.concat([newdata, appenddata], ignore_index=True)
    
    for index, row in data.iterrows():
        number = newdata.index[newdata['mac_hash']==row['mac_hash']][0]
        columntag=str(row['sniffer_loc'])
        newdata.at[number, columntag] = '1'

    newdata=newdata.loc[:,['mac_hash','1','2','3','4','5','6','7','8','9','10','11','12','13','14']]
    newdata=newdata[~newdata.mac_hash.str.contains("NaN")]
    newdata=newdata.fillna('0')
    #newdata = newdata.reset_index(drop=True)

    return newdata

testdataset=datapreprocess(testdataset)
testdataset = testdataset.reset_index(drop=True)
testdataset

mac_hash  1  2  3  4  5  6  7  8  9 10 11 12 13  \
0     f7e1b13eb50d989553d9bab2235a02e6  0  1  1  0  0  0  0  0  0  0  0  1  0   
1     bc57555f20709e7efda866f4636df8f6  0  1  1  1  1  0  1  1  1  1  1  1  1   
2     e34cbb4924a2ae73e73301ff3f7270fa  0  0  1  0  1  1  0  0  0  0  0  0  0   
3     1de801eef6272807ebcd0be707ba9995  0  1  1  1  1  1  1  1  1  0  1  1  0   
4     bd8f5de6a59c078064938b71aa3e1055  0  0  0  0  0  0  0  0  0  0  0  1  0   
...                                ... .. .. .. .. .. .. .. .. .. .. .. .. ..   
1386  a3a539edc2df183bb9725110ced88721  0  1  0  0  0  0  0  0  0  0  0  0  0   
1387  9d05e643d82f44305ad7ab8e04a9c755  0  0  0  1  0  0  0  0  1  0  1  1  1   
1388  a74d4fd585937277f7fc131ae8be61a5  0  1  0  1  0  1  1  0  1  1  1  0  1   
1389  cacf92cf30cc0cf81063f267e7513b66  0  1  0  1  0  1  0  1  1  1  0  0  0   
1390  0fd06086be3a241a360306ce70ac4224  0  1  0  1  0  0  0  0  0  0  0  0  0   

     14  
0     0  
1     0  
2     0  
3     0  
4     1  
...  ..  
1386  0  
1387  0  
1388  0  
1389  0  
1390  0  

[1391 rows x 15 columns]

### Predict the result using your model

In [17]:
# X_test: feature
testdata = testdataset.drop('mac_hash', axis=1)
testdata=testdata.astype('int64')

In [18]:
# Prediction
predicted =onehotencoder.inverse_transform(model.predict(testdata)).flatten()
predicted

44/44 [==============================] - 0s 696us/step


array([0, 4, 1, ..., 2, 2, 0], dtype=int64)

In [19]:
# Generate submit.csv
submit = pd.DataFrame({
    "mac_hash":testdataset['mac_hash'],
    "label":pd.Series(predicted).astype('int64')
})
submit=submit.sort_values(by=['mac_hash'], ascending=True)
submit = submit.reset_index(drop=True)
submit.to_csv('submit.csv', index = False)
submit

mac_hash  label
0     004c76bfe9f7b20f3651e0bcbd8eee9e      1
1     0084303d531091816387956ea26b86eb      1
2     00872a1ee635c2619454350b96b94103      4
3     00a5f69fc7719ba29c7b202f292dbafb      4
4     00ad41313f1f637e079c1a9d9a857d12      4
...                                ...    ...
1386  ff6883d04de66c6d4859928ec9d3f642      0
1387  ff7b41d3b33fb7d0b6704f1093173c40      4
1388  ffa5692a2dac937f456280c433961ea6      0
1389  ffcaf0f29c6bede7b5cd9bc807c31715      1
1390  ffd0c4403c3c44dcbe4772821b032252      4

[1391 rows x 2 columns]